# Run OpenMM on Google Colab


**<span style="color:#A03;font-size:14pt">
&#9888; WARNING
</span>**
> The installation procedure below easily breaks. It might not work when you try to reproduce this.

The purpose of this notebook is to show how to run an OpenMM simulation on [Google Colab](https://colab.research.google.com/). This typically consists of the following steps:

1. Instal OpenMM.
2. Upload data files to the cloud.
3. Run the simulation.
4. Download the results.

Do not run this notebook locally on your laptop. Upload it to [Google Colab](https://colab.research.google.com/) or copy-paste the code below into cells in a new notebook on [Google Colab](https://colab.research.google.com/).

In [ ]:
# Install OpenMM
import sys
print(sys.version)
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
!bash Miniconda3-py*.sh -bfp /usr/local
!conda config --set always_yes yes
!conda config --add channels conda-forge
!conda create -n openmm python=3.7 cudatoolkit=10.0 git jupyterlab numpy pandas scipy matplotlib ipympl rdkit openbabel openmm mdtraj pymbar pdbfixer parmed openff-toolkit openmoltools openmmforcefields
sys.path.append('/usr/local/envs/openmm/lib/python3.7/site-packages')
import openmm.testInstallation
openmm.testInstallation.main()

In [ ]:
# Import all the stuff we need
from google.colab import files
from openmm import *
from openmm.app import *
from openmm.unit import *

In [ ]:
# Upload the file alanine-dipeptide.pdb
uploaded = files.upload()

In [ ]:
# Copy the code from notebook2, example 3
pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('traj3.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(100000)

In [ ]:
# Download the results.
# This is terrible slow for large files, such as traj3.dcd (29MB)
files.download('init3.pdb')
files.download('scalars3.csv')
files.download('traj3.dcd')